In [23]:
# Importing notebook dependencies
%run FinanceTab.ipynb
%run twitter_sentiment_predict.ipynb
%run news_sentiment_predict.ipynb

# Initial imports
#  import os
#  import pandas as pd
import requests

from datetime import datetime
from datetime import timedelta, date
from dateutil.parser import parse
#  import matplotlib.pyplot as plt

# import interactive panels libraries
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, CustomJS, MultiSelect, CheckboxGroup, NumeralTickFormatter, HoverTool
from bokeh.models.widgets import Tabs, Panel

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/albertkong/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Error During Twitter Authentication


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/albertkong/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
############################################################
"""
    Get Predictions Function
"""
############################################################

def get_four_predictions(ticker = "MSFT", search_word = "microsoft"):

    """
    Technical Analysis, Fundamental Analysis, Twitter Sentiment, Article Sentiment
    Notebooks return the next 5 predicted price percent changes of a selected stock and save them into dataframes

    """
    
    # Run the Jupyter notebooks with user variables
    reg, info = trainFinModel(ticker)
    ta_predicted = predictPrices(ticker, reg, info)
    print("Financial Completed")
    twitter_predicted = tweet_sentiment_main_function(ticker, search_word)  
    print("Twitter Completed")
    news_predicted = news_sentiment_main_function(ticker, search_word)
    print("News Completed")
    
    avg_combined_predictions = combine_average(ta_predicted, twitter_predicted, news_predicted)

    return ta_predicted, twitter_predicted, news_predicted, avg_combined_predictions
    
    
############################################################
"""
    Combine and Average Functions
"""
############################################################
    
def combine_average(ta_df, twitter_df, news_df):   
    
    last_close = ta_df["close"][0]/(1+ta_df["DR"][0])
    ta_df.drop(columns = "close", inplace = True)
    
    # Aligning Indices
    twitter_df.reset_index(drop = True, inplace = True)
    twitter_df.set_index(ta_df.index, inplace = True)
    
    news_df.reset_index(drop = True, inplace = True)
    news_df.set_index(ta_df.index, inplace = True)
    
    # Joins all the prediction dataframes into a single dataframe
    combined_df = pd.concat([ta_df, twitter_df, news_df], axis = "columns", join = "inner")
    
    # Averaging across the rows (axis = 1) of the combined dataframe
    avg_combined_predictions = pd.DataFrame(combined_df.mean(axis = 1), columns = ["Pred Returns"])
    
    # Calculating Prices from the Last Close
    avg_combined_predictions["Pred Close"] = -1.0
    
    for i in range(avg_combined_predictions.shape[0]):
        if i == 0:
            avg_combined_predictions["Pred Close"][i] = last_close*(1+avg_combined_predictions["Pred Returns"][i])
        else:
            avg_combined_predictions["Pred Close"][i] = avg_combined_predictions["Pred Close"][i-1]*(1+avg_combined_predictions["Pred Returns"][i])

    
    return avg_combined_predictions

############################################################
"""
    Trade Recommendation Function
    
    A recommendation of 1.0 is to buy, -1.0 is to sell, 0.0 is to hold
"""
############################################################

def trade_reco(ticker, avg_combined_predictions):
    
    # Getting technical signals recommendation
    sig_rec = getSignalRec(ticker)
    
    # Calculating prediction recommendation
    pred_rec = 0.0
    open_price = avg_combined_predictions["Pred Close"].head(1)[0]
    print(open_price)
    close_price = avg_combined_predictions["Pred Close"].tail(1)[0]
    print(close_price)
    
    if close_price > open_price:
        pred_rec = 1.0
    elif close_price < open_price:
        pred_rec = -1.0
    
    # Sentiment recommendation
    sent_rec = 0.0 ## @Kris or Satheesh, how will you be making recommendations from sentiment?
    
    sum_rec = sig_rec + pred_rec + sent_rec
    
    if sum_rec > 0:
        trade_rec = "Buy"
    elif sum_rec < 0:
        trade_rec = "Sell"
    else:
        trade_rec = "Hold"
        
    return trade_rec


############################################################
"""
    Define Panel Prediction Plot
"""
############################################################

def price_prediction_graph(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    # Parse Column data
    source = ColumnDataSource(data=df)
    # Create Toolbar Selection
    select_tools = [HoverTool(), 'pan', 'wheel_zoom', 'tap', 'reset']
    
    tooltips = [
            ('Date', '@Date{%F}'),
            ('Adj. Close. Price', '@AdjClosePrice{$00.00}'),
               ]
    
    formatters = {
            '@Date' : 'datetime'
    }
    
    # Create the figure object
    predicted_fig = figure(
        x_axis_type='datetime',
        title=("Predicted Price Returns"),
        plot_height=500, plot_width=900,
        x_axis_label='Date', y_axis_label='Predicted Returns',
        toolbar_location='below',
        tools = select_tools
    )
    
    # Draw the figure
    predicted_fig.line(
        x='Date',
        y="predicted_change",
        source=source,
        color='green', 
        width=0.75, 
        legend_label='Price',
    )
       
    predicted_fig.legend.location = 'top_left'
    predicted_fig.add_tools(HoverTool(tooltips=tooltips, formatters=formatters))
    predicted_pp_pane = pn.pane.Bokeh(predicted_fig)
    
    return predicted_pp_pane


############################################################
"""
    Define & store user input variables
"""
############################################################

def ticker_input_store(user_input):
    ticker_input = ticker_input.value
    return ticker_input
    
def search_input_store(user_input):
    search_input = search_input.value
    return search_input
    

In [9]:
############################################################
"""
    Main Function
    Gets all predictions and stores them into dataframes.
    Uses those dataframes and returns the panes for the panel dashboard
"""
############################################################
def get_all_predictions(ticker, search_word):
    # gets predictions and saves them to dataframes
    ta_df, twitter_df, news_df, avg_combined_predictions = get_four_predictions(ticker, search_word)

    ############################################################
    """
        Creating The Panes
    """
    ############################################################ 
    
    # Creating the Price Prediction Pane
    avg_predicted_price_pane = price_prediction_graph(avg_combined_predictions)
    
    #Creating the Decision Recommendation Pane
    trade_reco_pane = trade_reco(ticker, avg_combined_predictions)
    
    # Creating the Individual Prediction Panes
    ta_predicted_pane = price_prediction_graph(ta_df)
    twitter_predicted_pane = price_prediction_graph(twitter_df)
    news_predicted_pane = price_prediction_graph(news_df)
    
    #Creating latest article & Tweet Panes
    #latest_article_pane = 
    #random_tweet_pane = 
    
    return avg_predicted_price_pane, trade_reco_pane, ta_predicted_pane, twitter_predicted_pane, news_predicted_pane

In [14]:
# Create Panel Dashboard
ticker = "MSFT"
search_word = "microsoft"
avg_predicted_price_pane, trade_reco_pane, ta_predicted_pane, twitter_predicted_pane, news_predicted_pane = get_all_predictions(ticker, search_word)

Financial Completed


Rate limit reached. Sleeping for: 180


Twitter Completed
News Completed


Loading BokehJS ...

239.4611781410372
379.541643448075


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [24]:
welcome_column = pn.Column(
    "# Welcome To The $4A Dashboard",
    "### We aim to maximize retail investors’ alpha over the long term by helping them make more informed decisions.",
    
    
    "#### Welcome to the asset analysis dashboard. Use the tabs above to navigate through the different sections."
    "Use the input below to see how your ideas and assets compare with the benchmarks.",
    
    "### Input the Stock Ticker and one search word you wish to view:",
    "##### *(Use ALL CAPS)*",
#     ticker_input = pn.widgets.TextInput(name='Stock Ticker:', placeholder='Enter a single ticker symbol.'),
#     ticker_input.on_submit(ticker = ticker_input.value),
    ticker,
#     search_input = pn.widgets.TextInput(name='Sentiment Search', placeholder='Enter a single search term for sentiment.'),
#     search_input.on_submit(search_word = search_input.value),
    search_word,

    #ticker = ticker_input.value,
    #search_word = search_input.value,
    
#     button = pn.widgets.Button(name='Predict', button_type='primary'),
#     #button.on_click(ticker = ticker_input.value, search_word = search_input.value)
#     button.on_click((avg_predicted_price_pane, trade_reco_pane, ta_predicted_pane, twitter_predicted_pane, news_predicted_pane = get_all_predictions(ticker, search_word))),
    
    "# Predicted Price Returns",
    avg_predicted_price_pane,
    
    " # The Recommendation",
    trade_reco_pane
)

individual_predictions = pn.Column(
    "# Predictions of Individual Analysis",
    
    "### Technical Analysis Predictions",
    ta_predicted_pane,
    "### Twitter Sentiment Analysis Predictions",
    twitter_predicted_pane,
    "### News Sentiment Analysis Predictions",
    news_predicted_pane,
)

# news_tweets = pn.Column(
#      "# Latest news article and random tweet about your stock:",
#     
#     "### Read The Latest News:",
#     latest_article_pane,
#     "### A Random Tweet For You:",
#     random_tweet_pane,
# )

finance_tab = getFinanceTab(ticker)

predictions_dashboard = pn.Tabs(
    (
        "Welcome",
        welcome_column
    ),
    (
        "Individual Analysis Predictions",
        individual_predictions
    ),
#     (
#         "Latest News",
#         news_tweets
#     ),
    (
        "Financial Information",
        finance_tab
    )
)

predictions_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] Markdown(str)
        [5] Markdown(str)
        [6] Markdown(str)
        [7] Markdown(str)
        [8] Bokeh(Figure)
        [9] Markdown(str)
        [10] Str(float)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Bokeh(Figure)
        [3] Markdown(str)
        [4] Bokeh(Figure)
        [5] Markdown(str)
        [6] Bokeh(Figure)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)
            [2] HoloViews(Curve)
            [3] HoloViews(NdOverlay)
        [1] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] HoloViews(Table)
            [3] Markdown(str)
            [4] HoloViews(Table)

In [25]:
# show the dashboard with the data embedded

predictions_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] Markdown(str)
        [5] Markdown(str)
        [6] Markdown(str)
        [7] Markdown(str)
        [8] Bokeh(Figure)
        [9] Markdown(str)
        [10] Str(float)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Bokeh(Figure)
        [3] Markdown(str)
        [4] Bokeh(Figure)
        [5] Markdown(str)
        [6] Bokeh(Figure)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)
            [2] HoloViews(Curve)
            [3] HoloViews(NdOverlay)
        [1] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] HoloViews(Table)
            [3] Markdown(str)
            [4] HoloViews(Table)

In [ ]:
"""
To Run code in Terminal

First need to run predictions_dashboard.servable()

then in terminal, run
panel serve dashboard.ipynb --show
"""